In [1]:
%%writefile study_agent.py
# -*- coding: utf-8 -*-
# Agente de Estudos Interativo

from dataclasses import dataclass        # <- corrigido: dataclasses
from typing import Dict, Protocol
import random

class Tool(Protocol):
    name: str
    def execute(self, input_text: str) -> str: ...

@dataclass
class TimerTool:
    name: str = "timer"
    def execute(self, input_text: str) -> str:
        return f"[TIMER] Iniciando foco de 25 minutos para: {input_text}\nDica: use o cronômetro."

@dataclass
class QuizTool:
    name: str = "quiz"
    def execute(self, input_text: str) -> str:
        n1, n2 = random.randint(5,10), random.randint(10,15)
        return (f"[QUIZ] Crie um algoritmo que leia dois vetores A({n1}) e B({n2}) "
                "e exiba os elementos NÃO repetidos (sem usar Set/HashSet).")

@dataclass
class LinkTool:
    name: str = "link"
    def execute(self, input_text: str) -> str:
        tema = input_text.lower()
        if "vetor" in tema: return "[LINK] https://exemplo.edu/vetores"
        if "poo" in tema:   return "[LINK] https://exemplo.edu/poo"
        return "[LINK] https://exemplo.edu/"

@dataclass
class ExplainTool:
    name: str = "explain"
    glossario: Dict[str,str] = None
    def __post_init__(self):
        if self.glossario is None:
            self.glossario = {
                "vetor": "Sequência de elementos acessados por índice.",
                "poo": "Paradigma orientado a objetos.",
                "pomodoro": "Técnica de estudo em blocos de 25min + pausa."
            }
    def execute(self, input_text: str) -> str:
        for k,v in self.glossario.items():
            if k in input_text.lower():
                return f"[EXPLAIN] {k}: {v}"
        return "[EXPLAIN] Tema não encontrado."

@dataclass
class StudyAgent:
    tools: Dict[str, Tool]
    def __init__(self):
        self.tools = {"timer": TimerTool(), "quiz": QuizTool(), "link": LinkTool(), "explain": ExplainTool()}
    def perceive(self, msg:str)->str: return msg.strip().lower()
    def decide(self, text:str)->str:
        if "pomodoro" in text or "foco" in text: return "timer"
        if "quiz" in text or "exercício" in text or "exercicio" in text: return "quiz"
        if "link" in text or "material" in text or "teoria" in text: return "link"
        if "o que é" in text or "o que e" in text or "explique" in text: return "explain"
        if "vetor" in text or "vetores" in text: return "quiz+link"
        return "desconhecido"
    def act(self, msg:str)->str:
        t = self.perceive(msg); intent = self.decide(t)
        if intent=="timer": return self.tools["timer"].execute(msg)
        if intent=="quiz": return self.tools["quiz"].execute(msg)
        if intent=="link": return self.tools["link"].execute(msg)
        if intent=="explain": return self.tools["explain"].execute(msg)
        if intent=="quiz+link":
            return self.tools["quiz"].execute(msg)+"\n"+self.tools["link"].execute(msg)
        return "Não entendi."


Writing study_agent.py


In [2]:
from study_agent import StudyAgent

agent = StudyAgent()
print("=== Agente de Estudos (interativo) ===")
print("Digite 'sair' para encerrar.\n")

while True:
    msg = input("Você: ")
    if msg.strip().lower() == "sair":
        print("Agente: Até logo! 👋")
        break
    print("Agente:", agent.act(msg), "\n")


=== Agente de Estudos (interativo) ===
Digite 'sair' para encerrar.



KeyboardInterrupt: Interrupted by user

In [ ]:
from study_agent import StudyAgent

agent = StudyAgent()
historico = []   # lista para guardar as interações

print("=== Agente de Estudos (interativo) ===")
print("Digite 'sair' para encerrar.\n")

while True:
    msg = input("Você: ")
    if msg.strip().lower() == "sair":
        print("Agente: Até logo! 👋")
        break

    resposta = agent.act(msg)
    print("Agente:", resposta, "\n")

    # salva no histórico
    historico.append({"usuario": msg, "agente": resposta})

# --- relatório ao final ---
print("\n📑 Relatório da Sessão:")
for i, troca in enumerate(historico, start=1):
    print(f"{i}. Você: {troca['usuario']}")
    print(f"   Agente: {troca['agente']}\n")

In [ ]:
from study_agent import StudyAgent
from datetime import datetime
import csv

agent = StudyAgent()
historico = []

print("=== Agente de Estudos (interativo) ===")
print("Digite 'sair' para encerrar.\n")

while True:
    try:
        msg = input("Você: ")
    except EOFError:
        # Colab às vezes lança EOF ao parar a célula
        break

    if msg.strip().lower() == "sair":
        print("Agente: Até logo! 👋")
        break

    resposta = agent.act(msg)
    print("Agente:", resposta, "\n")

    # Guarda no histórico
    historico.append({"usuario": msg, "agente": resposta})

# ----- Salvar relatório ao final -----
ts = datetime.now().strftime("%Y%m%d-%H%M%S")
txt_path = f"relatorio_sessao_{ts}.txt"
csv_path = f"relatorio_sessao_{ts}.csv"

# TXT
with open(txt_path, "w", encoding="utf-8") as f:
    f.write("📑 Relatório da Sessão (Agente de Estudos)\n\n")
    for i, troca in enumerate(historico, start=1):
        f.write(f"{i}. Você: {troca['usuario']}\n")
        f.write(f"   Agente: {troca['agente']}\n\n")

# CSV
with open(csv_path, "w", encoding="utf-8", newline="") as f:
    w = csv.writer(f)
    w.writerow(["turno", "usuario", "agente"])
    for i, troca in enumerate(historico, start=1):
        w.writerow([i, troca["usuario"], troca["agente"]])

print(f"\nArquivos salvos: {txt_path} e {csv_path}")


from google.colab import files

# ... (código do agente e histórico) ...

print(f"\nArquivos salvos: {txt_path} e {csv_path}")

# baixar automaticamente
files.download(txt_path)
files.download(csv_path)